# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name:  Terry Konkin  
  
https://github.com/TKonkin/article-summarizer

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

In [2]:
from collections import Counter
import pickle
import requests
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [7]:
# This cell is only run when the command is not successful in the terminal (occassionally).
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.0 MB/s eta 0:00:12
     -- ------------------------------------- 0.8/12.8 MB 1.1 MB/s eta 0:00:11
     ---- ----------------------------------- 1.3/12.8 MB 1.4 MB/s eta 0:00:09
     ----- ---------------------------------- 1.8/12.8 MB 1.9 MB/s eta 0:00:06
     ------- -------------------------------- 2.4/12.8 MB 2.0 MB/s eta 0:00:06
     --------- ------------------------------ 3.1/12.8 MB 2.2 MB/s eta 0:00:05
     ----------- ---------------------------- 3.7/12.8 MB 2.2 MB/s eta 0:00:05
     ------------- -------------------------- 4.2/12.8 MB 2.1 MB/s eta 0:00:05
     ------------- -------------------------- 4.2/12.8 MB 2.1 MB/s eta 0:00:05
     ---------------- ----------------------- 5.2/12.8 MB 2.2 MB/s eta 0:00:04
     ------------------ --------------------- 6.0/12.8 MB 2.3 MB/s

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

In [ ]:
url = "https://www.cnbc.com/2025/07/11/goldman-sachs-autonomous-coder-pilot-marks-major-ai-milestone.html"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html5lib')

article = soup.find('article')
article_html = str(article)

file_path = r'C:\Projects\article-summarizer\goldman-saks-ai-powered-new-employee.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(article_html, f)

print(f"Article HTML saved to {file_path}")

2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

In [11]:
# polarity score only

file_path = r"C:\Projects\article-summarizer\goldman-saks-ai-powered-new-employee.pkl"

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")

html_content = data[0]  
soup = BeautifulSoup(html_content, 'html.parser')

for tag in soup(['script', 'style']):
    tag.decompose()

article_text = soup.get_text(separator='\n', strip=True)

doc = nlp(article_text)

print(f"Article Polarity: {doc._.blob.polarity:.3f}")



Article Polarity: 0.000


In [12]:
# number of sentences

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")

file_path = r"C:\Projects\article-summarizer\goldman-saks-ai-powered-new-employee.pkl"

with open(file_path, 'rb') as f:
    data = pickle.load(f)

html_content = data[0]

soup = BeautifulSoup(html_content, 'html.parser')
for tag in soup(['script', 'style']):
    tag.decompose()
article_text = soup.get_text(separator='\n', strip=True)

doc = nlp(article_text)

print(f"Number of Sentences: {len(list(doc.sents))}")

Number of Sentences: 1


3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?